# Main Libraries for the Project

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# Reading the Dataset

In [ ]:
trainData = pd.read_csv("/Users/jeremyperez/Jupyter/NSL-KDD/KDDTrain+.csv") 
testData = pd.read_csv("/Users/jeremyperez/Jupyter/NSL-KDD/KDDTest+.csv")
trainData

# Getting the Dependent and independent Variables

In [ ]:
#Getting the Dependent and independent Variables
X = trainData.iloc[:,:-2].values # Get all the rows and all the clums except all the colums - 2
Y = trainData.iloc[:,41]# Get all the rows and the colum number 42
attacks = trainData.iloc[:,41] #Get only the attacks column

# Encoding Categorical Data

In [ ]:
# Encoding the categorical data
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
#We use dummy Encoding to pervent the machine learning don't atribute the categorical data in order. 
# Encoding the Independent Variable
 
labelencoder_X1 = LabelEncoder()
X[:, 1] = labelencoder_X1.fit_transform(X[:, 1])
labelencoder_X1 = LabelEncoder()
X[:, 2] = labelencoder_X1.fit_transform(X[:, 2])
labelencoder_X1 = LabelEncoder()
X[:, 3] = labelencoder_X1.fit_transform(X[:, 3])

# Encoding the Dependent Variable
labelencoder_y = LabelEncoder()
Y = labelencoder_y.fit_transform(Y)

# Scaling the data Stardar Scaling 

In [ ]:
#Feature Scaling
#We scaled the data to all fall within the range []
from sklearn.preprocessing import StandardScaler
standarscaler = StandardScaler()
X =  standarscaler.fit_transform(X)

# Elbow Method 

In [ ]:
#Elbow method to find the best number of culster
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++',max_iter = 300,n_init = 10,random_state = 0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()
#5 clusters 

# KMeans

In [ ]:
#Applying K-mea(n_clusters = 5)
kmeans = KMeans(n_clusters = 5, init = 'k-means++',max_iter = 300,n_init = 10,random_state = 0)
y_kmeans = kmeans.fit(X)
y_kmeans.labels_

# Analyzing Results by Crosstab

In [ ]:
pd.crosstab(attacks,y_kmeans.labels_)

# Analyzing Results by Graph

In [ ]:
#Visual representation of the clusters
plt.scatter(X[y_kmeans ==0,0],X[y_kmeans == 0,1], s = 21, c = 'red', label = 'cluster1')
plt.scatter(X[y_kmeans ==1,0],X[y_kmeans == 1,1], s = 21, c = 'yellow', label = 'cluster2')
plt.scatter(X[y_kmeans ==2,0],X[y_kmeans == 2,1], s = 21, c = 'cyan', label = 'cluster3')
plt.scatter(X[y_kmeans ==3,0],X[y_kmeans == 3,1], s = 21, c = 'orange', label = 'cluster4')
plt.scatter(X[y_kmeans ==4,0],X[y_kmeans == 4,1], s = 21, c = 'black', label = 'cluster5')
plt.scatter(kmeans.cluster_centers_[:, 0],kmeans.cluster_centers_[:, 1],s = 300, c = 'purple', label = 'Centroids')
plt.title('Clusters of Attacks')
plt.xlabel('Numbers of Attacks')
plt.ylabel('Types of Attacks')
plt.legend()
plt.show()

# DBSCAN 

In [ ]:
from sklearn.cluster import DBSCAN

# Create three gaussian blobs to use as our clustering data.
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

X = StandardScaler().fit_transform(X)

###############################################################################
# Scikit-learn implementation of DBSCAN
#

print 'Runing scikit-learn implementation...'
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
skl_labels = db.labels_

# Scikit learn uses -1 to for NOISE, and starts cluster labeling at 0. I start
# numbering at 1, so increment the skl cluster numbers by 1.
for i in range(0, len(skl_labels)):
    if not skl_labels[i] == -1:
        skl_labels[i] += 1


###############################################################################
# Did we get the same results?

num_disagree = 0

# Go through each label and make sure they match (print the labels if they 
# don't)
for i in range(0, len(skl_labels)):
    if not skl_labels[i] == my_labels[i]:
        print 'Scikit learn:', skl_labels[i], 'mine:', my_labels[i]
        num_disagree += 1

if num_disagree == 0:
    print 'PASS - All labels match!'
else:
    print 'FAIL -', num_disagree, 'labels don\'t match.'